# Setup

In [1]:
# Hardware-aware device selection
import torch

def get_device() -> str:
    """
    Returns the optimal device string for running PyTorch models, prioritizing CUDA (NVIDIA GPUs),
    Apple Silicon MPS (Metal), and falling back to CPU. Optionally checks for AMD ROCm/HIP.

    Returns
    ----------
    - device: str
        Name of the device to use: "cuda", "mps", "cpu", or "hip".

    Notes
    ----------
    - On NVIDIA GPUs, "cuda" is used for hardware acceleration.
    - On Apple Silicon, "mps" uses Metal Performance Shaders, providing hardware acceleration.
    - On AMD, "hip" (ROCm) may be available, but this is uncommon.
    - If no GPU is available, it defaults to "cpu".
    Raises
    ----------
    - None
    """
    # Check for CUDA (NVIDIA GPUs)
    if torch.cuda.is_available():
        return "cuda"
    # or MPS (Apple Silicon)
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    # or ROCm (AMD)
    if hasattr(torch.version, "hip") and torch.version.hip is not None:
        return "hip"
    # Fallback to CPU
    return "cpu"

device = get_device()
print(f"Using device: {device}")


Using device: mps


# Building the emotionRegressor

This class loads both transformers, runs inference, and outputs the 35d mood vector for a single input string. Handles device placement and memory cleanup.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class EmotionRegressor:
    """
    Loads and manages transformer model for emotional analysis, producing a 28-dimensional mood vector for a given text input.
    
    Models:
    ----------
    - SamLowe/roberta-base-go_emotions: Predicts confidence scores for 28 discrete emotions (GoEmotions).

    Parameters
    ----------
    - individual_emotion_model_name: str
        HuggingFace model name for the 28d emotion regressor (default: "SamLowe/roberta-base-go_emotions").
        - device: str or None
        Device to use for inference ("cuda", "mps", "cpu", etc). If None, selects best available.

    Methods
    -------
    - predict(text: str) -> np.ndarray
        Returns a 28d mood vector for a given text.
    - cleanup() -> None
        Frees model memory for safe re-initialization.
    """
    def __init__(self,
                 individual_emotion_model_name: str = "SamLowe/roberta-base-go_emotions",
                 device: str = None):
        """
        Initialize the EmotionRegressor and load both models on the appropriate device.

        Parameters
        ----------
        - individual_emotion_model_name: str
            Name or path for the 28-emotion model.

        - device: str or None
            Device for inference.

        Returns
        -------
        - None

        Raises
        -------
        - RuntimeError: if model loading fails or device is invalid.
        """
        # Load SamLowe model for individual emotions
        self.device = device or get_device()
        self.individual_emotion_tokenizer = AutoTokenizer.from_pretrained(individual_emotion_model_name)
        self.individual_emotion_model = AutoModelForSequenceClassification.from_pretrained(individual_emotion_model_name).to(self.device)
        self.individual_emotion_model.eval()

    def predict(self, text: str) -> np.ndarray:
        """
        Generates a 28-dimensional mood vector for the given input text.

        Parameters
        ----------
        - text: str
            The text to analyze for emotion content.

        Returns
        -------
        - mood_vector: np.ndarray, shape (28,)
            Concatenated vector: [28d GoEmotions scores]

        Raises
        -------
        - ValueError: if the text input is empty or not a string.
        - RuntimeError: if inference fails due to hardware or model issues.
        """
        # Validate input
        if not isinstance(text, str) or not text.strip():
            raise ValueError("Input text must be a non-empty string.")
        
        # Lowe model output (28d)
        inputs = self.individual_emotion_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            individual_emotion_logits = self.individual_emotion_model(**inputs).logits
            individual_emotion_probs = torch.sigmoid(individual_emotion_logits).cpu().numpy().flatten()  # (28,)
            
        # Combine into 35d vector
        mood_vector = np.concatenate([individual_emotion_probs])
        return mood_vector

    def cleanup(self) -> None:
        """
        Releases GPU/CPU memory by deleting model weights.

        Returns
        -------
        - None
        """
        # Delete models to free memory
        del self.individual_emotion_model
        import gc
        # Run garbage collection to free up memory
        gc.collect()
        # Clear CUDA cache if using GPU
        if self.device == "cuda":
            import torch
            torch.cuda.empty_cache()


# Demo of 28-dimensional mood_vector

## Hard-coded example

In [11]:
# Initialize and demo
emotion_regressor = EmotionRegressor(device=device)
sample_text = "ugh. like, im just so OVER everything right now. i have like, three midterms next week and a huge group project that my group members rNT even helping with. i stayed up till 4 am trying to finISH something for it but like, im just SO tired all the time. and my brain feels like mush. 😵‍💫 i just cant focus on anythinG. i keep staring at my textbook but the words just blur together."
mood_vector = emotion_regressor.predict(sample_text)
print("28d mood_vector:", mood_vector)
print("Shape:", mood_vector.shape)
emotion_regressor.cleanup()


28d mood_vector: [2.80239899e-03 5.84837422e-03 2.65013836e-02 6.07926965e-01
 1.78276524e-02 1.31448160e-03 4.62995330e-03 1.89981493e-03
 7.16498680e-03 4.01560247e-01 7.64833987e-02 2.34261509e-02
 1.14092352e-02 2.55096215e-03 1.22402806e-03 5.31175407e-04
 1.28757767e-03 2.61986558e-03 1.43351580e-03 4.59351484e-03
 3.81162507e-03 9.43467370e-04 4.12532315e-02 1.70239829e-03
 1.76575605e-03 3.49067971e-02 3.82844801e-03 1.19789466e-01]
Shape: (28,)


## Tinkering with output display format

In [ ]:
import pandas as pd

# These should be the order of the GoEmotions and Ekman categories used by your models
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

def pretty_print_mood_vector(mood_vector):
    """
    Prints the 28d mood vector in a readable format using pandas DataFrame.
    """
    if len(mood_vector) != 28:
        raise ValueError("Mood vector must be length 28")
    data = {**{f"{e}": v for e, v in zip(GOEMOTIONS_LABELS, mood_vector[:28])}}
    df = pd.DataFrame([data])
    print(df.T.rename(columns={0: 'confidence'}))  # .T to flip to (label, value) per row

# Usage
pretty_print_mood_vector(mood_vector)




                confidence
admiration        0.002802
amusement         0.005848
anger             0.026501
annoyance         0.607927
approval          0.017828
caring            0.001314
confusion         0.004630
curiosity         0.001900
desire            0.007165
disappointment    0.401560
disapproval       0.076483
disgust           0.023426
embarrassment     0.011409
excitement        0.002551
fear              0.001224
gratitude         0.000531
grief             0.001288
joy               0.002620
love              0.001434
nervousness       0.004594
optimism          0.003812
pride             0.000943
realization       0.041253
relief            0.001702
remorse           0.001766
sadness           0.034907
surprise          0.003828
neutral           0.119789


# live use (single-input)

In [5]:
emotion_regressor = EmotionRegressor(device=device)
user_input = input("Enter text: ")
mood_vector = emotion_regressor.predict(user_input)


In [6]:

print(user_input)

i'm a little upset to have to lose the second transformer, it had a much bigger wealth of datasets that it was trained on. but i have to make peace with the fact that its output was too broad, and not very useful for my purposes. hopefully the single-transformer model will be as powerful as the dual-transformer model i've left behind.


In [7]:
pretty_print_mood_vector(mood_vector)


                confidence
admiration        0.006564
amusement         0.001623
anger             0.004582
annoyance         0.064064
approval          0.019741
caring            0.013333
confusion         0.005940
curiosity         0.002808
desire            0.044165
disappointment    0.646095
disapproval       0.042262
disgust           0.003413
embarrassment     0.004060
excitement        0.002479
fear              0.004360
gratitude         0.002509
grief             0.004135
joy               0.006755
love              0.003109
nervousness       0.009303
optimism          0.471453
pride             0.001867
realization       0.028679
relief            0.005792
remorse           0.014383
sadness           0.077667
surprise          0.003924
neutral           0.061537


In [8]:
print(mood_vector)

[0.00656432 0.0016226  0.00458184 0.0640636  0.01974069 0.01333318
 0.0059402  0.00280816 0.04416487 0.6460954  0.04226154 0.00341327
 0.00406021 0.00247896 0.00435981 0.00250942 0.00413452 0.00675495
 0.00310854 0.00930327 0.47145328 0.00186655 0.02867904 0.00579177
 0.01438257 0.07766729 0.0039245  0.06153668]


In [9]:
list(mood_vector)

[0.0065643177,
 0.0016225978,
 0.004581841,
 0.0640636,
 0.019740686,
 0.013333182,
 0.005940196,
 0.002808163,
 0.04416487,
 0.6460954,
 0.042261545,
 0.0034132705,
 0.004060205,
 0.00247896,
 0.0043598125,
 0.002509423,
 0.00413452,
 0.0067549516,
 0.0031085385,
 0.009303273,
 0.47145328,
 0.0018665511,
 0.028679036,
 0.005791767,
 0.014382573,
 0.07766729,
 0.003924499,
 0.061536685]

In [10]:
len(mood_vector)

28